In [ ]:
import os
import torch
from torch import optim, nn, utils, Tensor
import pandas as pd
import torch.nn.functional as F
import pytorch_lightning as pl
from torch.utils.data import Dataset
import numpy as np
torch.manual_seed(0)

In [ ]:
from numpy import double
device = "cuda" if torch.cuda.is_available() else "cpu"

class SimpleNN(pl.LightningModule):
    def __init__(self):
        super().__init__()
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(104 * 22, 500)
        self.fc2 = nn.Linear(500, 100)
        self.fc3 = nn.Linear(100, 20)
        self.fc4 = nn.Linear(20, 5)
        self.fc5 = nn.Linear(5, 2)
        self.soft = nn.Softmax(dim = 0)
    
    def forward(self, x):
        x = torch.flatten(x)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = self.fc5(x)
        return x

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        # x = self.soft(x)
        lossFunc = nn.CrossEntropyLoss()
        if y == "1":
            y = torch.tensor([0,1]).float()
        else:
            y = torch.tensor([1,0]).float()
        print(self.soft(logits), y)
        loss = lossFunc(logits, y)
        return loss

    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(), lr = 1e-3)
        return optimizer


In [ ]:
simpleNN = SimpleNN().to(device).double()
print(simpleNN)

In [ ]:
directory = 'processedDataset/'
X = list()
Y = list()
for filename in os.listdir(directory)[0:320]:
    f = os.path.join(directory, filename)
    X.append(torch.load(f))
    if filename.split('-')[2].replace(".pt","") == '1':
        Y.append(torch.tensor([0,1]).float())
    else:
        Y.append(torch.tensor([1,0]).float())

In [ ]:
print(Y)
Y[1] == "1"

In [ ]:
from numpy import double


class stockDataset(Dataset):
    def __init__(self, X, Y):
        super().__init__()
        self.labels = Y
        self.data = X
    def __len__(self):
        return len(self.data)
    def __getitem__(self, index):
        print(self.labels[index])
        return self.data[index], self.labels[index]

In [ ]:
dataset = stockDataset(X,Y)

In [ ]:
train, test = torch.utils.data.random_split(dataset, [320,0])

In [ ]:
train_loader = torch.utils.data.DataLoader(train, 1, shuffle = True)
# test_loader = torch.utils.data.DataLoader(test, 1, shuffle = True)

In [ ]:
trainer = pl.Trainer(limit_train_batches=100, max_epochs=100)
trainer.fit(model=simpleNN, train_dataloaders=train_loader)

In [ ]:
# load checkpoint
checkpoint = "./lightning_logs/version_13/checkpoints/epoch=4-step=500.ckpt"
simpleNN = SimpleNN.load_from_checkpoint(checkpoint).double()

# choose your trained nn.Module
simpleNN.eval()

In [ ]:
sf = nn.Softmax(dim = 0)
simpleNN(X[0])

In [ ]:
print(Y[0])

In [ ]:
for i in range(len(X)):
    print(simpleNN(X[i]), Y[i])

In [ ]:
lossFunc = nn.CrossEntropyLoss()
input = torch.tensor([0.54,0.78])
target = torch.tensor([0,1]).float()
print(input)
print(target)
output = lossFunc(input,target)
print(output)

In [ ]:
simpleNN(torch.randn([104, 22], dtype=float))

In [ ]:
# import torch
# import torch.nn as nn
# import torch.nn.functional as F
# import numpy as np

# class Model(nn.Module):
#     """
#     Just one Linear layer
#     """
#     def __init__(self, configs):
#         super(Model, self).__init__()
#         self.seq_len = configs.seq_len
#         self.pred_len = configs.pred_len
#         self.Linear = nn.Linear(self.seq_len, self.pred_len)
#         # Use this line if you want to visualize the weights
#         # self.Linear.weight = nn.Parameter((1/self.seq_len)*torch.ones([self.pred_len,self.seq_len]))

#     def forward(self, x):
#         # x: [Batch, Input length, Channel]
#         x = self.Linear(x.permute(0,2,1)).permute(0,2,1)
#         return x # [Batch, Output length, Channel]